In [3]:
#Import needed libraries and get data
%matplotlib inline

import pandas as pd
import numpy as np
import re
import nltk
import urllib
from matplotlib import pyplot as plt
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier

train = pd.read_table('train.tsv')
train["Length"] = train['Phrase'].apply(lambda x: len(x.split()))
ordered = train.sort(['SentenceId', 'Length'], ascending=[1, 0])
train_sentences = train.groupby('SentenceId').first().reset_index()


max_length = max(ordered['Length'])
bins = [0, 1, max_length/15, max_length/4, max_length/2, max_length]
group_names = ['SingleWord', 'SmallPhrase', 'Phrase', 'LongPhrase', 'Sentences']
categories = pd.cut(ordered['Length'], bins, labels=group_names)
ordered['categories'] = pd.cut(ordered['Length'], bins, labels=group_names)


sentences = ordered[ordered['categories'] == 'Sentences'].reset_index()
longphrase = ordered[ordered['categories'] == 'LongPhrase'].reset_index()
phrase = ordered[ordered['categories'] == 'Phrase'].reset_index()
smallphrase = ordered[ordered['categories'] == 'SmallPhrase'].reset_index()
singleword = ordered[ordered['categories'] == 'SingleWord'].reset_index()
print bins[::-1]
print len(sentences), len(longphrase), len(phrase), len(smallphrase), len(singleword)
ordered

/home/kiki/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
/home/kiki/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:16: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


[52, 26, 13, 3, 1, 0]
4193 20141 68215 46979 16531


,PhraseId,SentenceId,Phrase,Sentiment,Length,categories
0,1,1,A series of escapades demonstrating the adage ...,1,37,Sentences
27,28,1,"is also good for the gander , some of which oc...",2,23,LongPhrase
28,29,1,"is also good for the gander , some of which oc...",2,22,LongPhrase
31,32,1,"good for the gander , some of which occasional...",2,20,LongPhrase
32,33,1,"for the gander , some of which occasionally am...",2,19,LongPhrase
33,34,1,"the gander , some of which occasionally amuses...",1,18,LongPhrase
38,39,1,some of which occasionally amuses but none of ...,2,15,LongPhrase
1,2,1,A series of escapades demonstrating the adage ...,2,14,LongPhrase
5,6,1,of escapades demonstrating the adage that what...,2,12,Phrase
43,44,1,occasionally amuses but none of which amounts ...,2,12,Phrase


In [4]:
files=['negative.txt','positive.txt']
path='http://www.unc.edu/~ncaren/haphazard/'
for file_name in files:
    urllib.urlretrieve(path+file_name,file_name)

In [5]:
neg = open("negative.txt").read().split()
neg

['abandoned',
 'abandonment',
 'aberration',
 'aberration',
 'abhorred',
 'abhorrence',
 'abhorrent',
 'abhorrently',
 'abhors',
 'abhors',
 'abject',
 'abjectly',
 'abnormal',
 'abominable',
 'abominably',
 'abomination',
 'abrasive',
 'abrupt',
 'absence',
 'absent-minded',
 'absurd',
 'absurdity',
 'absurdly',
 'absurdness',
 'abuse',
 'abuse',
 'abuses',
 'abuses',
 'abusive',
 'abysmal',
 'abysmally',
 'abyss',
 'accidental',
 'accountable',
 'accursed',
 'accusation',
 'accusation',
 'accusations',
 'accusations',
 'accuses',
 'accusing',
 'accusingly',
 'acerbic',
 'acerbically',
 'acrid',
 'acridly',
 'acridness',
 'acrimonious',
 'acrimoniously',
 'acrimony',
 'adamant',
 'adamantly',
 'addict',
 'addiction',
 'admonition',
 'adrift',
 'adulterated',
 'adulteration',
 'adversarial',
 'adverse',
 'adversity',
 'affectation',
 'affliction',
 'afflictive',
 'affront',
 'afraid',
 'against',
 'aggravating',
 'aggravation',
 'aggression',
 'aggression',
 'aggressiveness',
 'aggress

In [6]:
pos = open("positive.txt").read().split()
pos

['abidance',
 'abidance',
 'abilities',
 'ability',
 'able',
 'above',
 'above-average',
 'abundant',
 'abundance',
 'acceptance',
 'acceptable',
 'accessible',
 'acclaim',
 'acclaimed',
 'accolade',
 'accolades',
 'accommodative',
 'accomplishment',
 'accomplishments',
 'accordance',
 'accordantly',
 'accurate',
 'accurately',
 'achievable',
 'achievement',
 'achievements',
 'acknowledgement',
 'active',
 'acumen',
 'adaptable',
 'adaptability',
 'adaptive',
 'adept',
 'adeptly',
 'adequate',
 'adherence',
 'adherent',
 'adhesion',
 'admirable',
 'admirer',
 'admirable',
 'admirably',
 'admiration',
 'admiring',
 'admiringly',
 'admission',
 'admission',
 'adorable',
 'adored',
 'adorer',
 'adoring',
 'adoringly',
 'adroit',
 'adroitly',
 'adulatory',
 'advanced',
 'advantage',
 'advantage',
 'advantageous',
 'advantages',
 'advantages',
 'adventure',
 'adventure',
 'adventuresome',
 'adventurism',
 'adventurous',
 'advice',
 'advice',
 'advisable',
 'advocacy',
 'affable',
 'affabili

In [7]:
def review_to_words( raw_review ):
    letters_only = re.sub("[^a-zA-Z]", " ", raw_review) 
    words = letters_only.lower().split()                             
    stops = set(stopwords.words("english"))                  
    meaningful_words = [w for w in words if not w in stops]   
    return( " ".join( meaningful_words ))   

In [8]:
print stopwords.words("english")

[u'i', u'me', u'my', u'myself', u'we', u'our', u'ours', u'ourselves', u'you', u'your', u'yours', u'yourself', u'yourselves', u'he', u'him', u'his', u'himself', u'she', u'her', u'hers', u'herself', u'it', u'its', u'itself', u'they', u'them', u'their', u'theirs', u'themselves', u'what', u'which', u'who', u'whom', u'this', u'that', u'these', u'those', u'am', u'is', u'are', u'was', u'were', u'be', u'been', u'being', u'have', u'has', u'had', u'having', u'do', u'does', u'did', u'doing', u'a', u'an', u'the', u'and', u'but', u'if', u'or', u'because', u'as', u'until', u'while', u'of', u'at', u'by', u'for', u'with', u'about', u'against', u'between', u'into', u'through', u'during', u'before', u'after', u'above', u'below', u'to', u'from', u'up', u'down', u'in', u'out', u'on', u'off', u'over', u'under', u'again', u'further', u'then', u'once', u'here', u'there', u'when', u'where', u'why', u'how', u'all', u'any', u'both', u'each', u'few', u'more', u'most', u'other', u'some', u'such', u'no', u'nor', u

In [9]:
clean_review = review_to_words(train_sentences["Phrase"][0])
print clean_review

series escapades demonstrating adage good goose also good gander occasionally amuses none amounts much story


In [10]:
num_reviews_sent = sentences["Phrase"].size
clean_sent_reviews = []

for i in xrange(0, num_reviews_sent):
    clean_sent_reviews.append( review_to_words(sentences["Phrase"][i]))
    
num_reviews_long = longphrase["Phrase"].size
clean_long_reviews = []

for i in xrange(0, num_reviews_long):
    clean_long_reviews.append( review_to_words(longphrase["Phrase"][i]))
    
num_reviews_p = phrase["Phrase"].size
clean_p_reviews = []

for i in xrange(0, num_reviews_p):
    clean_p_reviews.append( review_to_words(phrase["Phrase"][i]))
    
num_reviews_small = smallphrase["Phrase"].size
clean_small_reviews = []

for i in xrange(0, num_reviews_small):
    clean_small_reviews.append( review_to_words(smallphrase["Phrase"][i]))

num_reviews_single = singleword["Phrase"].size
clean_single_reviews = []

for i in xrange(0, num_reviews_single):
    clean_single_reviews.append( review_to_words(singleword["Phrase"][i]))

In [ ]:
singleword['POS'] = 'N/A'
for i in xrange(0, num_reviews_single):
    singleword['POS'][i] = nltk.pos_tag(singleword['Phrase'][i])
singleword

In [52]:
vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 4000) 

# fit_transform() does two functions: First, it fits the model
# and learns the vocabulary; second, it transforms our training data
# into feature vectors. The input to fit_transform should be a list of 
# strings.
sent_data_features = vectorizer.fit_transform(clean_sent_reviews)
sent_data_features = sent_data_features.toarray()

long_data_features = vectorizer.fit_transform(clean_long_reviews)
long_data_features = long_data_features.toarray()

p_data_features = vectorizer.fit_transform(clean_p_reviews)
p_data_features = p_data_features.toarray()

small_data_features = vectorizer.fit_transform(clean_small_reviews)
small_data_features = small_data_features.toarray()

single_data_features = vectorizer.fit_transform(clean_single_reviews)
single_data_features = single_data_features.toarray()

In [53]:
print sent_data_features.shape, long_data_features.shape, p_data_features.shape, small_data_features.shape, single_data_features.shape

(4193, 4000) (20141, 4000) (68215, 4000) (46979, 4000) (16531, 4000)


In [54]:
vocab = vectorizer.get_feature_names()
print len(vocab), vocab

4000 [u'aaa', u'able', u'absorbed', u'accident', u'act', u'acted', u'acting', u'action', u'actor', u'actress', u'adolescent', u'adrenaline', u'adult', u'adults', u'adventure', u'advised', u'affectation', u'affirming', u'african', u'age', u'aged', u'agency', u'agenda', u'ages', u'aggrandizing', u'aggressive', u'ago', u'ah', u'ahead', u'aid', u'aids', u'air', u'already', u'altman', u'america', u'american', u'americans', u'amused', u'amusing', u'angle', u'angst', u'animal', u'animated', u'anne', u'annie', u'anti', u'arm', u'around', u'art', u'artificiality', u'arts', u'asleep', u'assess', u'assured', u'attention', u'audacious', u'audience', u'auto', u'autobiographical', u'avant', u'average', u'awake', u'award', u'aware', u'awareness', u'awe', u'baby', u'back', u'bad', u'badly', u'baked', u'balanced', u'ball', u'ballistic', u'banderas', u'bang', u'banter', u'bare', u'barn', u'baseball', u'based', u'bear', u'beast', u'beat', u'beer', u'behaving', u'behind', u'believe', u'belly', u'beloved',

In [55]:
# Sum up the counts of each vocabulary word
dist = np.sum(sent_data_features, axis=0)

# For each, print the vocabulary word and the number of times it 
# appears in the training set
for tag, count in zip(vocab, dist):
    print count, tag

26 aaa
37 able
5 absorbed
8 accident
7 act
10 acted
5 acting
10 action
10 actor
5 actress
5 adolescent
8 adrenaline
5 adult
6 adults
13 adventure
6 advised
9 affectation
5 affirming
8 african
5 age
5 aged
12 agency
17 agenda
10 ages
74 aggrandizing
104 aggressive
38 ago
82 ah
12 ahead
12 aid
13 aids
7 air
37 already
5 altman
6 america
8 american
6 americans
9 amused
9 amusing
11 angle
12 angst
6 animal
8 animated
6 anne
4 annie
15 anti
8 arm
13 around
5 art
6 artificiality
15 arts
18 asleep
16 assess
10 assured
11 attention
7 audacious
8 audience
10 auto
5 autobiographical
5 avant
40 average
7 awake
7 award
10 aware
10 awareness
9 awe
11 baby
7 back
7 bad
5 badly
14 baked
5 balanced
5 ball
12 ballistic
10 banderas
11 bang
14 banter
6 bare
8 barn
10 baseball
17 based
57 bear
17 beast
18 beat
27 beer
136 behaving
5 behind
34 believe
7 belly
24 beloved
5 bender
6 bending
9 bent
14 best
27 bettany
67 better
7 beyond
5 bible
6 big
27 bigger
6 bill
5 bio
10 bit
20 bitten
19 biz
5 black
32 bl

In [56]:
# Initialize a Random Forest classifier with 100 trees
forest_sent = RandomForestClassifier(n_estimators = 20) 

# Fit the forest to the training set, using the bag of words as 
# features and the sentiment labels as the response variable
#
# This may take a few minutes to run
forest_sent = forest_sent.fit(sent_data_features, sentences["Sentiment"])

In [57]:
# Initialize a Random Forest classifier with 100 trees
forest_long = RandomForestClassifier(n_estimators = 20) 

# Fit the forest to the training set, using the bag of words as 
# features and the sentiment labels as the response variable
#
# This may take a few minutes to run
forest_long = forest_long.fit(long_data_features, longphrase["Sentiment"])

In [59]:
# Initialize a Random Forest classifier with 100 trees
forest_p = RandomForestClassifier(n_estimators = 20) 

# Fit the forest to the training set, using the bag of words as 
# features and the sentiment labels as the response variable
#
# This may take a few minutes to run
forest_p = forest_p.fit(p_data_features, phrase["Sentiment"])

In [60]:
# Initialize a Random Forest classifier with 100 trees
forest_small = RandomForestClassifier(n_estimators = 20) 

# Fit the forest to the training set, using the bag of words as 
# features and the sentiment labels as the response variable
#
# This may take a few minutes to run
forest_small = forest_small.fit(small_data_features, smallphrase["Sentiment"])

In [61]:
# Initialize a Random Forest classifier with 100 trees
forest_single = RandomForestClassifier(n_estimators = 20) 

# Fit the forest to the training set, using the bag of words as 
# features and the sentiment labels as the response variable
#
# This may take a few minutes to run
forest_single = forest_single.fit(single_data_features, singleword["Sentiment"])

In [62]:
print "done"

done


In [66]:
test = pd.read_table('test.tsv')
test["Length"] = test['Phrase'].apply(lambda x: len(x.split()))
ordered = test.sort(['SentenceId', 'Length'], ascending=[1, 0])

max_length = max(ordered['Length'])
bins = [0, 1, max_length/15, max_length/4, max_length/2, max_length]
group_names = ['SingleWord', 'SmallPhrase', 'Phrase', 'LongPhrase', 'Sentences']
categories = pd.cut(ordered['Length'], bins, labels=group_names)
ordered['categories'] = pd.cut(ordered['Length'], bins, labels=group_names)


sentences = ordered[ordered['categories'] == 'Sentences'].reset_index()
longphrase = ordered[ordered['categories'] == 'LongPhrase'].reset_index()
phrase = ordered[ordered['categories'] == 'Phrase'].reset_index()
smallphrase = ordered[ordered['categories'] == 'SmallPhrase'].reset_index()
singleword = ordered[ordered['categories'] == 'SingleWord'].reset_index()


num_reviews_sent = sentences["Phrase"].size
clean_sent_reviews = []

print "Cleaning and parsing the test set movie reviews...\n"
for i in xrange(0, num_reviews_sent):
    clean_sent_reviews.append( review_to_words(sentences["Phrase"][i]))
    if( (i+1) % 10000 == 0 ):
        print "Review %d of %d\n" % (i+1, len(ordered))
    
num_reviews_long = longphrase["Phrase"].size
clean_long_reviews = []

for i in xrange(0, num_reviews_long):
    clean_long_reviews.append( review_to_words(longphrase["Phrase"][i]))
    if( (i+1) % 10000 == 0 ):
        print "Review %d of %d\n" % (i+1, len(ordered))
        
num_reviews_p = phrase["Phrase"].size
clean_p_reviews = []

for i in xrange(0, num_reviews_p):
    clean_p_reviews.append( review_to_words(phrase["Phrase"][i]))
    if( (i+1) % 10000 == 0 ):
        print "Review %d of %d\n" % (i+1, len(ordered))
    
num_reviews_small = smallphrase["Phrase"].size
clean_small_reviews = []

for i in xrange(0, num_reviews_small):
    clean_small_reviews.append( review_to_words(smallphrase["Phrase"][i]))
    if( (i+1) % 10000 == 0 ):
        print "Review %d of %d\n" % (i+1, len(ordered))

num_reviews_single = singleword["Phrase"].size
clean_single_reviews = []

for i in xrange(0, num_reviews_single):
    clean_single_reviews.append( review_to_words(singleword["Phrase"][i]))
    if( (i+1) % 10000 == 0 ):
        print "Review %d of %d\n" % (i+1, len(ordered))

print "Applying tree to sentences...\n"        
sent_data_features = vectorizer.fit_transform(clean_sent_reviews)
sent_data_features = sent_data_features.toarray()
result_sent = forest_sent.predict(sent_data_features)

print "Applying tree to long phrases...\n" 
long_data_features = vectorizer.fit_transform(clean_long_reviews)
long_data_features = long_data_features.toarray()
result_long = forest_long.predict(long_data_features)

print "Applying tree to mid phrases...\n" 
p_data_features = vectorizer.fit_transform(clean_p_reviews)
p_data_features = p_data_features.toarray()
result_p = forest_p.predict(p_data_features)

print "Applying tree to small phrases...\n" 
small_data_features = vectorizer.fit_transform(clean_small_reviews)
small_data_features = small_data_features.toarray()
result_small = forest_small.predict(small_data_features)

print "Applying tree to single words...\n" 
single_data_features = vectorizer.fit_transform(clean_single_reviews)
single_data_features = single_data_features.toarray()
result_single = forest_single.predict(single_data_features)

#print result_sent, result_long, result_small, result_single

# Copy the results to a pandas dataframe with an "id" column and
# a "sentiment" column
#output = pd.DataFrame(data={"PhraseId":test_sentences["PhraseId"], "Sentiment":result})

# Use pandas to write the comma-separated output file
#output.to_csv( "rotten_tomatoes_model.csv", index=False, quoting=3 )
print "done predicting"

Cleaning and parsing the test set movie reviews...

Review 10000 of 66292

Review 20000 of 66292

Review 10000 of 66292

Review 10000 of 66292

Applying tree to sentences...

Applying tree to long phrases...

Applying tree to mid phrases...

Applying tree to small phrases...

Applying tree to single words...

done predicting


/home/kiki/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:3: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  app.launch_new_instance()


In [75]:
sentences['Sentiment'] = result_sent.tolist()
longphrase['Sentiment'] = result_long.tolist()
phrase['Sentiment'] = result_p.tolist()
smallphrase['Sentiment'] = result_small.tolist()
singleword['Sentiment'] = result_single.tolist()
print len(sentences)+len(longphrase)+len(phrase)+len(smallphrase)+len(singleword)

66291


In [76]:
df = pd.merge(sentences, longphrase, how='outer')
df1 = pd.merge(df, phrase, how='outer')
df2 = pd.merge(df1,smallphrase, how='outer')
df3 = pd.merge(df2, singleword, how='outer')
df3 = df3.sort(['PhraseId']).reset_index()

/home/kiki/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:5: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [77]:
remaining = test[~test['PhraseId'].isin(df3['PhraseId'])].reset_index()
remaining['Sentiment'] = int(2)
df3 = pd.merge(df3, remaining, how='outer')
df3["PhraseId"] = df3["PhraseId"].apply(lambda x : int(x))
df3["Sentiment"] = df3["Sentiment"].apply(lambda x : int(x))
print len(df3)

66292


In [78]:
output = pd.DataFrame(data={"PhraseId":df3["PhraseId"], "Sentiment":df3['Sentiment']})

# Use pandas to write the comma-separated output file
output.to_csv( "rotten_tomatoes_model.csv", index=False, quoting=3 )